This is some pseudocode to illustrate the reasoning in [bayesianweaklensing.ipynb](bayesianweaklensing.ipynb))

In [5]:
def getBProb(b):
    # TODO: return probability for b (actually b/a fraction)
    return 1
    
def getSourceEllipticityProb(E1, E2):
    E = (E1**2+E2**2)**0.5
    b = (1-E)/(1+E)
    return 1/(math.pi * E * (E+1)**2) * getBProb(b)

def getRealImageEllipticityProb(e_img1, e_img2, g_1, g_2):
    if g_1**2 + g_2**2 <= 1:
        jacobian = (g_1**2 + g_2**2 - 1)**2/(e_img1**2*g_1**2 + e_img1**2*g_2**2 - 2*e_img1*g_1 + e_img2**2*g_1**2 + e_img2**2*g_2**2 - 2*e_img2*g_2 + 1)**2
        E1 = (-(e_img1 - g_1)*(e_img1*g_1 + e_img2*g_2 - 1) + (e_img2 - g_2)*(e_img1*g_2 - e_img2*g_1))/((e_img1*g_2 - e_img2*g_1)**2 + (e_img1*g_1 + e_img2*g_2 - 1)**2)
        E2 = -((e_img1 - g_1)*(e_img1*g_2 - e_img2*g_1) + (e_img2 - g_2)*(e_img1*g_1 + e_img2*g_2 - 1))/((e_img1*g_2 - e_img2*g_1)**2 + (e_img1*g_1 + e_img2*g_2 - 1)**2)        
    else:
        jacobian = (g_1**2 + g_2**2 - 1)**2/(e_img1**2 - 2*e_img1*g_1 + e_img2**2 - 2*e_img2*g_2 + g_1**2 + g_2**2)**2
        E1 = (-(e_img1 - g_1)*(e_img1*g_1 + e_img2*g_2 - 1) + (e_img2 - g_2)*(e_img1*g_2 - e_img2*g_1))/((e_img1 - g_1)**2 + (e_img2 - g_2)**2)
        E2 = -((e_img1 - g_1)*(e_img1*g_2 - e_img2*g_1) + (e_img2 - g_2)*(e_img1*g_1 + e_img2*g_2 - 1))/((e_img1 - g_1)**2 + (e_img2 - g_2)**2)

    return jacobian * getSourceEllipticityProb(E1, E2)

def getEllSamples(e, sigma):
    # This samples the gaussian at -3 sigma, -2 sigma, ... 3 sigma
    # Just as an illustration, these are the settings I'm using for testing now
    samplesAndWeights = [ ]
    for k in range(-3, 4):
        sample = e + k*sigma
        weight = math.exp(-0.5*((e-sample)/sigma)**2)
        samplesAndWeights.append([sample, weight])
    
    return samplesAndWeights

def getObservedImageEllipticityProb(e_obs1, e_obs2, sigma1, sigma2, g_1, g2):
    if sigma1 == 0 and sigma2 == 0:
        return getRealImageEllipticityProb(e_obs1, e_obs2, g_1, g_2)
    
    # Not zero, need to do that double integral, which we'll approximate using a double sum
    
    e1SamplesAndWeights = getEllSamples(e_obs1, sigma1)
    e2SamplesAndWeights = getEllSamples(e_obs2, sigma2)
    
    weightSum = 0
    probSum = 0
    for e_samp1, w1 in e1Samples:
        for e_samp2, w2 in e2Samples:
            probSum += w1*w2 * getRealImageEllipticityProb(e_samp1, e_samp2, g_1, g_2)
            weightSum += w1*w2
            
    return probSum/weightSum
            
def getObservedImageEllipticityProbForRedshift(e_obs1, e_obs2, sigma1, sigma2, z_obs, z_uncert):
    
    if z_obs != 0 and z_uncert == 0: # Redshift is known exactly
        g_1, g_2 = calculateReducedShearAtRedshift(z_obs)
        return getObservedImageEllipticityProb(e_obs1, e_obs2, sigma1, sigma2, g_1, g_2)
    
    # Unknown z, approximate the integral with a sum (in the code)
    weigthSum = 0
    probSum = 0
    
    # The getZValuesAndProbabilities function returns a list of (z, prob) tuples. Depending
    # on z_obs and z_uncert this can be either for a completely unknown redshift, or based
    # on a redshift known with a specific uncertainty
    
    for z, zprob in getZValuesAndProbabilities(z_lens, z_obs, z_uncert):
        g_1, g_2 = calculateReducedShearAtRedshift(z)
        
        probSum += weight * getObservedImageEllipticityProb(e_obs1, e_obs2, sigma1, sigma2, g_1, g_2)
        weightSum += weight
        
    return probSum/weight
        